# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Предпросмотр

**Imports**

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)

**Read data**

In [3]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Info**

In [4]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.7,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.6,покупка жилья
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля
2,0,-5623.4,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья
3,3,-4124.7,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.6,дополнительное образование
4,0,340266.1,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.1,сыграть свадьбу


In [6]:
df.tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21520,1,-4529.3,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.9,операции с жильем
21521,0,343937.4,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.8,сделка с автомобилем
21522,1,-2113.3,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.6,недвижимость
21523,3,-3112.5,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.1,на покупку своего автомобиля
21524,2,-1984.5,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.4,на покупку автомобиля


In [7]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Имеются пропуски в столбцах days_employed, total_income

In [8]:
unique_columns=['education','family_status','gender','income_type','purpose']
for column in unique_columns:
  print(column,'---',df[column].unique())

education --- ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
family_status --- ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
gender --- ['F' 'M' 'XNA']
income_type --- ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
purpose --- ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' '

In [9]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.0,19351.0,21525.0,21525.0,21525.0,21525.0,19351.0
mean,0.5,63046.5,43.3,0.8,1.0,0.1,167422.3
std,1.4,140827.3,12.6,0.5,1.4,0.3,102971.6
min,-1.0,-18388.9,0.0,0.0,0.0,0.0,20667.3
25%,0.0,-2747.4,33.0,1.0,0.0,0.0,103053.2
50%,0.0,-1203.4,42.0,1.0,0.0,0.0,145017.9
75%,1.0,-291.1,53.0,1.0,1.0,0.0,203435.1
max,20.0,401755.4,75.0,4.0,4.0,1.0,2265604.0


Имеются отрицательные значения в столбцах children,days_employed. Выбросы в столбце children, нулевые значения в столбце dob_years

In [10]:
df.loc[df['dob_years'] == 0,'dob_years'].count()

101

In [11]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

## Предобработка данных

### Обработка пропусков

Удалим строку с XNA

In [13]:
df.drop(df.loc[df['gender'] == 'XNA'].index, inplace=True)

Заменим значения в столбце children

In [14]:
df.loc[df['children'] == 20, 'children'] = 2 

In [15]:
df.loc[df['children'] == -1, 'children'] = 1 

Заменим нулевые значения возраста на медианные значения в группах по профессиям

In [16]:
mapping = pd.Series(df.groupby('income_type')['dob_years'].median()).to_dict()

In [17]:
df.loc[df['dob_years']==0,'dob_years']=df['income_type'].map(mapping)

In [18]:
df.loc[df['dob_years'] == 0,'dob_years'].count()

0

Заменим отрицательные значения в столбце days_emploed

In [19]:
df.loc[df['days_employed']>=df['dob_years']*365]['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

In [20]:
df['days_employed']=df['days_employed'].abs()

In [21]:
df.loc[df['days_employed']>=df['dob_years']*365,'dob_years'].count()

3445

Добавим новый столбец с группами по возрасту, в зависимости от значения квантилей.

In [22]:
quantiles = df['dob_years'].quantile([0.25, 0.5, 0.75]).tolist()

In [23]:
quantiles

[34.0, 43.0, 53.0]

In [24]:
def age_group(row):
  age = row['dob_years']
  if age<quantiles[0]:
    return '25%'
  elif age < quantiles[1]  :
    return '50%'
  elif age < quantiles[2]  :
    return '75%'
  else:
    return '100%'

Заменим некорректные даные на NaN

In [25]:
df['days_employed'].isna().sum()

2174

In [26]:
df.loc[(df['days_employed']>=df['dob_years']*365),'days_employed'] = None

In [27]:
df['days_employed'].isna().sum()

5619

In [28]:
df['age_group'] = df.apply(age_group, axis=1)

In [29]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.7,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.6,покупка жилья,50%
1,1,4024.8,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля,50%
2,0,5623.4,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья,25%
3,3,4124.7,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.6,дополнительное образование,25%
4,0,nan,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.1,сыграть свадьбу,100%


In [30]:
df['days_employed'].describe()

count   15905.0
mean     2353.0
std      2304.3
min        24.1
25%       756.3
50%      1630.0
75%      3157.7
max     18388.9
Name: days_employed, dtype: float64

Заменим пропущенные значения на медианные в зависимости от возрастной группы

In [31]:
mapping_days = pd.Series(df.groupby('age_group')['days_employed'].median()).to_dict()

In [32]:
mapping_days

{'100%': 2373.358619291148,
 '25%': 1134.3765732304414,
 '50%': 1778.8824573530192,
 '75%': 2105.631867843327}

In [33]:
df.loc[df['days_employed'].isna(),'days_employed']=df['age_group'].map(mapping_days)

Таким же образом обработаем пропуски в total_income

In [34]:
mapping_total = pd.Series(df.groupby('age_group')['total_income'].median()).to_dict()

In [35]:
df.loc[df['total_income'].isna(),'total_income']=df['age_group'].map(mapping_total)

In [36]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21524.0,21524.0,21524.0,21524.0,21524.0,21524.0,21524.0
mean,0.5,2301.3,43.5,0.8,1.0,0.1,165210.9
std,0.8,1992.1,12.2,0.5,1.4,0.3,97904.0
min,0.0,24.1,19.0,0.0,0.0,0.0,20667.3
25%,0.0,1025.6,34.0,1.0,0.0,0.0,107796.0
50%,0.0,2105.6,43.0,1.0,0.0,0.0,145904.5
75%,1.0,2518.3,53.0,1.0,1.0,0.0,195538.7
max,5.0,18388.9,75.0,4.0,4.0,1.0,2265604.0


In [37]:
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
age_group           0
dtype: int64

**Вывод**

1. В столбцах **days_employed и total_income** имеются отсутствующие данные, причем отсутствие данных повторяется в обоих столбцах, что говорит о неслучайности пропусков.
2. В столбце **days_employed** у пенсионеров и безработных указаны некорректные значения с пропусками, составляющие 100% от общего количества
3. В столбце **dob_years** так же присутствуют случайные пропуски, не зависящие от других данных выборки
4. XNA присутствует в столбце gender в одном экземпляре, и является полностью случайным.

Причины ошибочных данных в категории пенсиоры являются систематическими и возможно связаны с некорректной работой программного обеспечения сбора данных, в остальных случаях пропуски в таблицах трудового стажа и зп скорее всего являются результатом нежелания клиента указывать соответствующие данные.

### Замена типа данных

In [38]:
df['days_employed']=df['days_employed'].astype('int')

In [39]:
df['dob_years']=df['dob_years'].astype('int')

### Обработка дубликатов

In [40]:
df['education'] = df['education'].str.lower()

In [41]:
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [42]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [43]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4176
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [44]:
df.duplicated().sum()

71

In [45]:
df = df.drop_duplicates().reset_index(drop=True)

**Вывод**

В столбце **education** были обнаружены дубликаты, наличие которых связано с вводом повторяющихся слов в разных регистрах

### Лемматизация

In [46]:
lemmas = ' '.join(df['purpose'])

In [47]:
m = Mystem()
lemmas = Counter(m.lemmatize(lemmas))

In [48]:
lemmas.most_common()

[(' ', 55021),
 ('недвижимость', 6350),
 ('покупка', 5896),
 ('жилье', 4460),
 ('автомобиль', 4306),
 ('образование', 4013),
 ('с', 2918),
 ('операция', 2604),
 ('свадьба', 2324),
 ('свой', 2230),
 ('на', 2222),
 ('строительство', 1878),
 ('высокий', 1374),
 ('получение', 1314),
 ('коммерческий', 1311),
 ('для', 1289),
 ('жилой', 1230),
 ('сделка', 941),
 ('дополнительный', 906),
 ('заниматься', 904),
 ('подержать', 853),
 ('проведение', 768),
 ('сыграть', 765),
 ('сдача', 651),
 ('семья', 638),
 ('собственный', 635),
 ('со', 627),
 ('ремонт', 607),
 ('приобретение', 461),
 ('профильный', 436),
 ('подержанный', 111),
 ('\n', 1)]

In [49]:
purpose_words = ['жилье','автомобиль','образование','свадьба','недвижимость']

**Вывод**

Лемматизацию проводили на основе уникальных значений целей кредита. Выделили 5 основных групп -**'жилье','автомобиль','образование','свадьба','недвижимость'**, на основании которых добавили новый столбец для группировки целей кредитования

### Категоризация данных

In [50]:
def purpose_group(row):
  for word in purpose_words:
    if word in m.lemmatize(row):
      return word
  return 'other'

In [51]:
df['purpose_group'] = df['purpose'].apply(purpose_group)

In [52]:
quantiles_income = df['total_income'].quantile([0.25, 0.5, 0.75]).tolist()

In [53]:
quantiles_income

[107620.94810490453, 145904.52007216375, 195799.51488463086]

In [54]:
def income_group(row):
  income = row['total_income']
  if income <= quantiles_income[0]:
    return 'Доход до 108000'

  elif income <= quantiles_income[1]:
    return 'Доход до 146000%'

  if income <= quantiles_income[2]:
    return 'Доход  до 196000%'
  return 'Доход > 196000'

In [55]:
df['income_group'] = df.apply(income_group, axis=1)

## Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [56]:
children_debt = df.groupby('children').agg({'debt':['count', 'sum', 'mean']})

In [57]:
children_debt['rate']=100*(children_debt['debt']['sum']/children_debt['debt']['count'])

In [58]:
children_debt

debt            rate
          count   sum mean     
children                       
0         14090  1063  0.1  7.5
1          4855   445  0.1  9.2
2          2128   202  0.1  9.5
3           330    27  0.1  8.2
4            41     4  0.1  9.8
5             9     0  0.0  0.0

**Вывод**

Наименьшую задолженность по кредитам имеют клиенты без детей, с ростом количества детей наблюдается рост показателя задолженности, это может быть связано с увеличением финансовой нагрузки всвязи с затратами на детей, снижением семейных показателей зп за счет ухода в декрет. Так же следует отметить уменьшение обращений в кредитные организации с ростом количества детей.

## Общий вывод

С увеличением числа детей у клиентов возрастает долговая нагрузка, связанная с затратами на детей. Неженатые или находящиеся в гражданском браке клиенты чаще, чем женатые просрачивают кредиты, что может быть связано с гражданско-правовыми отношениями в семье(выплаты по кредитам осуществляются из общего бюджета семьи, а не из личных средств). Процент задолженностей по кредитам увеличивается с увеличением роста зп, это может быть связано с увеличением размера одобренного кредита, и, как следствие, увеличением долговой нагрузки. Дальнейшее увеличение зп может привести к финансовой устойчивости и уменьшению закредитованности. Кредиты на операции с недвижимостью просрачиваются реже потребительских кредитов Чаще всего просрачиваются на автокредиты и образование, что может быть связано с увеличением финансовой нагрузки(в случае эксплуатации авто), рисков потери работы в первые годы после окончания обучения, низкими зп выпускников и возрастающей финансовой нагрузкой(в случае образования).